In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
df = pd.read_csv('big_stat.csv')#pd.read_csv('BBGM_League_7_all_seasons_Average_Stats.csv')
df = df[(df.G*df.MP > 82)]
df.shape


In [ ]:
V = df[['Name','Ovr']].groupby('Name').max()

In [ ]:
(V >= 90).sum()

In [ ]:
vals = [76.9,63.9,56.2,50.8,46.6,43.2,40.3,37.8,35.6,33.6,31.8,30.2,28.7,27.3,26,24.8,23.6,22.6,21.5,20.6,19.7,18.8,18,17.2,16.4,15.6,14.9,14.3,13.6,13]
#df = df[df.Season >= 2030]

In [ ]:
vals2 = []
for i in range(30):
    agg = df[df['DraftPick'] == i+1].groupby('Name').sum()
    agg = agg.mean()
    #print(i,agg['WS'],agg['EWA'])
    vals2.append(agg['WS'])
#_ = plt.hist(['EWA'],20)
plt.plot(np.arange(30)+1,vals,label='NBA')
plt.plot(np.arange(30)+1,vals2,label='BBGM')
plt.xlabel('draft pick')
plt.xlim(1,30)
plt.ylabel('Mean Career Win Shares')
plt.legend()
plt.gca().set_facecolor('w')
#plt.gca().set_facecolor('w')
plt.savefig('draft.png',transparent=False)

In [ ]:
vals3 = []
valid_ages = set(list(range(19,36)))
for i in range(19,36):
    agg = df[df['Age'] == i]#.groupby('Name').sum()
    agg = agg.mean()
    #print(i,agg['WS'],agg['EWA'])
    vals3.append(agg['WS/48'])

In [ ]:
WS_REAL = defaultdict(list)
for YR in range(1980,2020):
    table = np.load('tables_{}.pkl'.format(YR))
    for team in table:
        stats = table[team]['advanced'][['Age','WS/48']]
        for item in stats.itertuples():
            WS_REAL[int(item[1])].append(item[2])

In [ ]:
vals4 = np.zeros(len(valid_ages))
for k,v in WS_REAL.items():
    if k in valid_ages:
        try:
            vals4[k-19] = np.array( [ _ for _ in v if type(_) == float ]).mean()
        except:
            print(np.array(v))

In [ ]:
plt.plot(range(19,36),vals4,label='NBA')
plt.plot(range(19,36),vals3,label='BBGM')
plt.xlabel('age')
plt.xlim(19,36)
plt.ylabel('Mean WS/48')
plt.legend()
plt.gca().set_facecolor('w')
#plt.gca().set_facecolor('w')
plt.savefig('ws48.png',transparent=False)

In [ ]:
dkeys = ['Hgt', 'Str', 'Spd', 'Jmp', 'End', 'Ins', 'Dnk', 'FT', '2Pt', '3Pt',
       'oIQ', 'dIQ', 'Drb', 'Pss', 'Reb']
new_ovr = []
nc = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])

new_ovr = np.array(df.iloc[:,-15:]) @ (nc/nc.sum())
#for row in df.iterrows():
#    new_ovr.append(sum([row[1][k]*v for k,v in zip(dkeys,nc)])/nc.sum())
#    break

plt.hist(new_ovr,100)
print(np.array(new_ovr).mean())
df.columns[-15:]

In [ ]:
df[df['+/-'] < -20]['Ovr']

In [ ]:
y = df.iloc[:,-18:-17] #45
#y.iloc[:,0] += (df.iloc[:,45]-0.074737)*50 #45
#y /=2 

X = df.iloc[:,-15:] # -17:-16
#X = X[[_ for _ in X.columns if _ != '+/-' ]]
X.columns,y.columns,y.shape

In [ ]:
_ = plt.hist(y.iloc[:,0],20)

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=True)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
#prescale_X = fexp.fit_transform(prescale_X)

In [ ]:
trials = 1
ts = []
prescale_X = (X-round(X.mean(0)))/round(X.std(0))
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    #clf = neural_network.MLPRegressor((32,4),'identity',solver='lbfgs',alpha=5e2,tol=1e-9)
    #clf = multioutput.MultiOutputRegressor(linear_model.SGDRegressor(penalty='l2',alpha=5e2,eta0=1e-6,tol=1e-12,max_iter=50,verbose=True))
    clf = multioutput.MultiOutputRegressor(linear_model.ElasticNetCV(cv=5,positive=True,fit_intercept=False))
    #clf = ensemble.ExtraTreesRegressor(8,criterion='mae',max_depth=3,verbose=1)
    #clf = multioutput.MultiOutputRegressor(svm.SVR())
    W = df.MP*df.G
    W /= W.mean()
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))

In [ ]:
ts = sorted(ts)[::1] # why not the biggest error
print(ts[0][0])
clf = ts[0][1]

In [ ]:
round(df.iloc[:,-15:].mean()),round(df.iloc[:,-15:].std())

In [ ]:
col_names = X.columns
#col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(y.columns):
    coeffs = clf.estimators_[i].coef_ 
    v = np.argsort(abs(coeffs))[::-1]
    print(c)
    coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v]
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('{:.2f} * {} + '.format(v,n),end='')
    print('| Var      | Coeff |')
    print('|----------|-------|')
    for v,n in sorted(coeffs2,reverse=True):
        print('|{:10s}|{:.2f}|'.format(n,100*v/coeffs.sum()))
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('\t{:25s}\t{:.2f}'.format(n,v))


In [ ]:
nc_M = []
for n,w in zip(col_names,clf.estimators_[0].coef_):
    if n == '1':
        continue
    w /= (clf.estimators_[0].coef_.sum())*0.01
    w = int(round(w))
    print(w,end=', ')
    nc_M.append(w)
nc_M = np.array(nc_M)

In [ ]:
round(X.std(0))

In [ ]:
coeff = clf.estimators_[i].coef_ 
coeff = np.round(coeff,2)
coeff[np.where(coeff <= 0)] = 0.01

pred = (X-round(X.mean(0)))/round(X.std(0)) @ coeff
MM = new_ovr.mean()
SS = new_ovr.std()/pred.std()
print(MM,SS)

new_ovr = np.array(new_ovr)
plt.hist(new_ovr,100)
plt.hist(pred*SS+MM,100,alpha=0.5)
plt.ylim(0,50)
new_ovr.mean(),new_ovr.std(),coeff

In [ ]:
GEN_YEAR = 1988
with open('rosters/auto_roster_{}.json'.format(GEN_YEAR),'r') as fp:
    base = json.load(fp)
players = base['players']

In [ ]:
dkeys = [_.lower() for _ in ['Hgt', 'Stre', 'Spd', 'Jmp', 'Endu', 'Ins', 'Dnk', 'FT', 'fg', 'tp',
       'oIQ', 'dIQ', 'Drb', 'Pss', 'Reb']]
nc = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])
xm = round(X.mean(0))
xs = round(X.std(0))

In [ ]:
player_df = []
for p in players:
    if not 'name' in p:
        p['name'] = p['firstName'] + ' ' + p['lastName']
    d = {'name':p['name']}
    d.update(p['ratings'][0])
    d['ovr_og'] = sum([d[k]*v for k,v in zip(dkeys,nc)])/nc.sum()
    d['ovr_mn'] = sum([(d[k]-m)*v/s for k,v,m,s in zip(dkeys,coeff,xm,xs)])*SS + MM
    #d['ovr_mnr'] = sum([d[k]*v for k,v in zip(dkeys,nc)])/np.maximum(0,nc3).sum()

    player_df.append(d)

In [ ]:
pdf = pd.DataFrame(player_df).set_index('name')
pdf['diff'] = pdf['ovr_mn'] - pdf['ovr_og']
#pdf['diffr'] = pdf['ovr_mnr'] - pdf['ovr_og']


In [ ]:
pdf.sort_values(['ovr_og'],0,False)


In [ ]:
pdf.sort_values(['ovr_mn'],0,False)


In [ ]:
pdf.sort_values(['diff'],0,False)


In [ ]:
_ = plt.hist(pdf['diff'],40)

In [ ]:
pdf['diff'].mean(),pdf['diff'].std()

In [ ]:
plt.figure(figsize=(10,10))
#plt.scatter(pdf['ovr_og'],pdf['ovr_mn'],s=11,alpha=0.7)
plt.scatter(new_ovr,pred*(SS)+MM+0.5,s=3,alpha=0.1)


plt.xlim(30,75)
plt.ylim(30,75)
plt.plot([30,75],[30,75],c='k')
plt.grid(True)